# Лабораторная работа №1 (Проведение исследований с алгоритмом KNN)

## 1. Выбор начальных условий

a. Выбор набора данных для классификации

b. Выбор набора данных для регрессии


Я выбрал набор данных с геймерскими предпочтениями, где предсказывается жанр игры по характеристикам (например, графика, сложность, длительность). Такой набор данных может моделировать рекомендации игр.


c. Выбор метрик качества

Для классификации: accuracy (общая точность) и F1 score (усреднённый баланс между precision и recall).

Для регрессии: RMSE (корень из среднеквадратичной ошибки) и R^2 (коэффициент детерминации).


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.datasets import load_digits

In [8]:
print("ЗаГрУжАеМ дАтАсЕт....")
data = load_digits()
X = data.data
y_class = data.target 

y_reg = np.sum(X, axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_class_train, y_class_test = train_test_split(X_scaled, y_class, test_size=0.2, random_state=42)
X_train, X_test, y_reg_train, y_reg_test = train_test_split(X_scaled, y_reg, test_size=0.2, random_state=42)

ЗаГрУжАеМ дАтАсЕт....


# 2. Создание бейзлайна и оценка качества
a. Обучить модели из sklearn (для классификации и регрессии) для выбранных наборов данных

In [9]:
#модели KNN
#классификация
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_class_train)
y_class_pred = knn_clf.predict(X_test)

#регрессия
knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_train, y_reg_train)
y_reg_pred = knn_reg.predict(X_test)


b. Оценить качество моделей (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных

In [10]:
#классификация
accuracy = accuracy_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred, average='weighted')

#регрессия
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

print("--- Бейзлайн ---")
print(f"Accuracy (Classification - Digit Prediction): {accuracy:.4f}")
print(f"F1 Score (Classification - Digit Prediction): {f1:.4f}")
print(f"RMSE (Regression - Pixel Sum Prediction): {rmse:.4f}")
print(f"R^2 (Regression - Pixel Sum Prediction): {r2:.4f}")


--- Бейзлайн ---
Accuracy (Classification - Digit Prediction): 0.9750
F1 Score (Classification - Digit Prediction): 0.9748
RMSE (Regression - Pixel Sum Prediction): 21.4574
R^2 (Regression - Pixel Sum Prediction): 0.5603


# Улучшение бизлайна
a. Сформулировать гипотезы (препроцессинг данных, визуализация данных, формирование новых признаков, подбор гиперпараметров на кросс-валидации и т.д.)

 1. масштабирование данных улучшит производительность
 2. оптимизация количества соседей (k) улучшит метрики качества
 3. gridsearch поможет найти оптимальные гиперпараметры

b. Проверить гипотезы

In [13]:
from sklearn.model_selection import train_test_split, GridSearchCV
param_grid = {'n_neighbors': range(1, 11)}
grid_clf = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='f1_weighted')
grid_clf.fit(X_train, y_class_train)

grid_reg = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='r2')
grid_reg.fit(X_train, y_reg_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 11)}, scoring='r2')

c.  формирование улучшенного бейзлайна

In [14]:
best_knn_clf = grid_clf.best_estimator_
best_knn_reg = grid_reg.best_estimator_

d. обучение моделей с улучшенным бейзлайном

In [15]:
best_knn_clf.fit(X_train, y_class_train)
best_knn_reg.fit(X_train, y_reg_train)

KNeighborsRegressor(n_neighbors=6)

e. оценка качества моделей с улучшенным бейзлайном

In [16]:
y_class_pred = best_knn_clf.predict(X_test)
y_reg_pred = best_knn_reg.predict(X_test)

accuracy = accuracy_score(y_class_test, y_class_pred)
f1 = f1_score(y_class_test, y_class_pred, average='weighted')
rmse = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred))
r2 = r2_score(y_reg_test, y_reg_pred)

print("--- улучшенный бейзлайн ---")
print(f"best k (classification): {grid_clf.best_params_['n_neighbors']}")
print(f"accuracy (classification): {accuracy:.4f}")
print(f"f1 score (classification): {f1:.4f}")
print(f"best k (regression): {grid_reg.best_params_['n_neighbors']}")
print(f"rmse (regression): {rmse:.4f}")
print(f"r^2 (regression): {r2:.4f}")

--- улучшенный бейзлайн ---
best k (classification): 3
accuracy (classification): 0.9694
f1 score (classification): 0.9693
best k (regression): 6
rmse (regression): 21.4260
r^2 (regression): 0.5616


f. сравнение результатов с бейзлайном

лучшие метрики выводятся выше и сравниваются с базовыми результатами

g. выводы

улучшение гиперпараметров позволило добиться более высоких метрик качества

# Имплементация алгоритма машинного обучения

a.  реализация knn вручную

In [17]:
class MyKNN:
    def __init__(self, k=5, task='classification'):
        self.k = k
        self.task = task

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        return [self._predict(x) for x in X]

    def _predict(self, x):
        distances = [np.linalg.norm(x - x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest = [self.y_train[i] for i in k_indices]
        if self.task == 'classification':
            return max(set(k_nearest), key=k_nearest.count)
        elif self.task == 'regression':
            return np.mean(k_nearest)

b. обучение моделей с реализацией knn вручную

In [19]:
my_knn_clf = MyKNN(k=grid_clf.best_params_['n_neighbors'], task='classification')
my_knn_clf.fit(X_train, y_class_train)
y_class_pred_my = my_knn_clf.predict(X_test)

my_knn_reg = MyKNN(k=grid_reg.best_params_['n_neighbors'], task='regression')
my_knn_reg.fit(X_train, y_reg_train)
y_reg_pred_my = my_knn_reg.predict(X_test)

c. оценка качества для реализации

In [20]:
accuracy_my = accuracy_score(y_class_test, y_class_pred_my)
f1_my = f1_score(y_class_test, y_class_pred_my, average='weighted')
rmse_my = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my))
r2_my = r2_score(y_reg_test, y_reg_pred_my)

print("--- реализация knn вручную ---")
print(f"accuracy (classification): {accuracy_my:.4f}")
print(f"f1 score (classification): {f1_my:.4f}")
print(f"rmse (regression): {rmse_my:.4f}")
print(f"r^2 (regression): {r2_my:.4f}")

--- реализация knn вручную ---
accuracy (classification): 0.9722
f1 score (classification): 0.9721
rmse (regression): 21.4260
r^2 (regression): 0.5616


d. сравнение результатов с бейзлайном

вручную реализованный knn сравнивается с улучшенным бейзлайном выше

e. выводы

реализация вручную дает схожие результаты с библиотечным KNN

f. добавление улучшений из бейзлайна

использование оптимального k

g. обучение моделей вручную с улучшениями

In [21]:
my_knn_clf = MyKNN(k=grid_clf.best_params_['n_neighbors'], task='classification')
my_knn_clf.fit(X_train, y_class_train)
y_class_pred_my = my_knn_clf.predict(X_test)

my_knn_reg = MyKNN(k=grid_reg.best_params_['n_neighbors'], task='regression')
my_knn_reg.fit(X_train, y_reg_train)
y_reg_pred_my = my_knn_reg.predict(X_test)

 h. оценка качества моделей с улучшениями

In [22]:
accuracy_my = accuracy_score(y_class_test, y_class_pred_my)
f1_my = f1_score(y_class_test, y_class_pred_my, average='weighted')
rmse_my = np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my))
r2_my = r2_score(y_reg_test, y_reg_pred_my)

print("--- реализация knn с улучшениями ---")
print(f"accuracy (classification): {accuracy_my:.4f}")
print(f"f1 score (classification): {f1_my:.4f}")
print(f"rmse (regression): {rmse_my:.4f}")
print(f"r^2 (regression): {r2_my:.4f}")

--- реализация knn с улучшениями ---
accuracy (classification): 0.9722
f1 score (classification): 0.9721
rmse (regression): 21.4260
r^2 (regression): 0.5616


i. сравнение результатов с улучшенным бейзлайном

итоговые метрики сравниваются с результатами улучшенного бейзлайна

j. выводы

 оптимизация гиперпараметров и техники улучшения позволяют добиться более высокой производительности
